# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('..\weatherPy\Output\city_data.csv')
weather_data.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Jamestown,90,US,1619755274,100,42.0970,-79.2353,46.004,5.66
1,1,Āsosa,100,ET,1619755431,78,10.0667,34.5333,63.266,1.79
2,3,Hobart,20,AU,1619755431,55,-42.8794,147.3294,68.000,2.57
3,4,Saint George,1,US,1619755432,17,37.1041,-113.5841,75.992,1.40
4,5,Sherbakul',0,RU,1619755432,35,54.6316,72.3963,51.800,5.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data[['Lat','Lng']]
humidity = weather_data['Humidity']

In [5]:
gmaps.configure(api_key=g_key)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=weather_data['Humidity'].max(), dissipating = False,point_radius=3)
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 5 mph.
#cloudiness less than 5.
#Humidity between 30 - 50
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
weather_data_filter_df = weather_data.loc[(weather_data['Humidity']>30)&(weather_data['Humidity']<=50)&(weather_data['Wind Speed']<5)&(weather_data['Cloudiness']<10)&(weather_data['Max Temp']>65)&(weather_data['Max Temp']<=85),:]
weather_data_filter_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
33,37,Abu Samrah,9,SY,1619755440,38,35.3029,37.1841,71.150,2.29
83,89,Cabo San Lucas,0,MX,1619755454,44,22.8909,-109.9124,71.006,0.45
178,191,Tongren,7,CN,1619755482,44,27.7172,109.1853,83.858,2.48
195,210,Codrington,0,AU,1619755486,35,-38.2667,141.9667,74.696,4.50
247,263,Buraidah,0,SA,1619755501,39,26.3260,43.9750,66.200,2.06
328,357,Wuning,0,CN,1619755524,45,29.2589,120.2311,80.780,4.18
419,454,Laguna,1,US,1619755550,37,38.4210,-121.4238,71.600,3.09
545,587,Abnūb,0,EG,1619755586,40,27.2696,31.1511,69.800,3.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#Create hotel_df dataframe, and store filtered locations to the new df
hotel_df=pd.DataFrame(weather_data_filter_df)
#Add a 'Hotel Name' column
hotel_df['Hotel Name'] =""
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
33,37,Abu Samrah,9,SY,1619755440,38,35.3029,37.1841,71.150,2.29,
83,89,Cabo San Lucas,0,MX,1619755454,44,22.8909,-109.9124,71.006,0.45,
178,191,Tongren,7,CN,1619755482,44,27.7172,109.1853,83.858,2.48,
195,210,Codrington,0,AU,1619755486,35,-38.2667,141.9667,74.696,4.50,
247,263,Buraidah,0,SA,1619755501,39,26.3260,43.9750,66.200,2.06,
328,357,Wuning,0,CN,1619755524,45,29.2589,120.2311,80.780,4.18,
419,454,Laguna,1,US,1619755550,37,38.4210,-121.4238,71.600,3.09,
545,587,Abnūb,0,EG,1619755586,40,27.2696,31.1511,69.800,3.09,


In [28]:
#Call google place api and return list of hotels based on the lat/lon pair of the cities in hotel_df
#Take the first result from the api call result and assign the hotel name to the hotel_df
#if no result found, then catch the exception and print out missing result message.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_type = "hotel"
target_radius = 5000
params = {
        "keyword": target_search,
        "radius":target_radius,
        "type": target_type,
        "key": g_key
    }

#loop through hotel_df and pass api calls
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    params['location']=target_coordinates

   
    try:
        #assemble url and make API request
        print(f"Retrieving Results for Index {index}: {target_coordinates}.")
        response = requests.get(base_url, params=params).json()
        # extract results
        results = response['results'][0]['name']

        print(f"Closest hotel near {target_coordinates} is {results}.")
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")    
    print("------------")

Retrieving Results for Index 33: 35.3029,37.1841.
Missing field/result... skipping.
------------
Retrieving Results for Index 83: 22.8909,-109.9124.
Closest hotel near 22.8909,-109.9124 is Bahia Hotel & Beach House.
------------
Retrieving Results for Index 178: 27.7172,109.1853.
Closest hotel near 27.7172,109.1853 is Tongren Grand Metro Park Hotel.
------------
Retrieving Results for Index 195: -38.2667,141.9667.
Closest hotel near -38.2667,141.9667 is Codrington Gardens Bed And Breakfast.
------------
Retrieving Results for Index 247: 26.326,43.975.
Closest hotel near 26.326,43.975 is Mövenpick Hotel Qassim.
------------
Retrieving Results for Index 328: 29.2589,120.2311.
Closest hotel near 29.2589,120.2311 is Fudu Hotel Yihai Road.
------------
Retrieving Results for Index 419: 38.421,-121.4238.
Closest hotel near 38.421,-121.4238 is Hilton Garden Inn Sacramento Elk Grove.
------------
Retrieving Results for Index 545: 27.2696,31.1511.
Missing field/result... skipping.
------------


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map - issue with tool tip display
markers = gmaps.marker_layer(locations, info_box_content=hotel_info,display_info_box=True)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))